In [12]:
#!pwsh 
dotnet build .\..\BinaryArchive00\BinaryArchive00.csproj -v quiet -o bin


Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:00.86


In [13]:
#!csharp

using System.IO;

string PathToArchives = File.ReadLines("PathToArchives").First();

In [14]:
#!csharp
#r "./bin/BinaryArchive00.dll"

using BinaryArchive00;

In [15]:
var filePaths = Directory.GetFiles(PathToArchives, "*.ca", SearchOption.AllDirectories);
var archives = filePaths.Select(path => ArchiveFile.Open(path)).ToArray();

var dv2 = archives.Single(archive => archive.FileName == "dv2.ca");
var dv2patch = archives.Single(archive => archive.FileName == "dv2patch.ca");
var music = archives.Single(archive => archive.FileName == "music.ca");
var music2 = archives.Single(archive => archive.FileName == "music2.ca");
var language = archives.Single(archive => archive.FileName == "language.ca");
var langpatch = archives.Single(archive => archive.FileName == "langpatch.ca");

In [16]:
#!csharp
#r "nuget:ConsoleTables"

using ConsoleTables;

public void PrintTable(string[] columns, object[][] rows)
{
    ConsoleTable table = new(columns);
    foreach (var row in rows)
        table.AddRow(row);
    table.Write();
}

public void SaveTable(string filePath, string[] columns, object[][] rows)
{
    Directory.CreateDirectory(Path.GetDirectoryName(filePath));

    var consoleOut = Console.Out;
    using FileStream fileStream = new(filePath, FileMode.Create, FileAccess.Write);
    using StreamWriter writer = new(fileStream) { AutoFlush = true };
    Console.SetOut(writer);

    ConsoleTable table = new(columns);
    foreach (var row in rows)
        table.AddRow(row);
    table.Write();

    Console.SetOut(consoleOut);
}

public void SaveEntry(ArchiveEntry entry, string filePath)
{
    Directory.CreateDirectory(Path.GetDirectoryName(filePath));
    File.WriteAllBytes(filePath, entry.Content);
}

bool AreEqual(byte[] array1, byte[] array2)
{
    if (array1 == array2)
        return true;

    if (array1 == null || array2 == null || array1.Length != array2.Length)
        return false;

    return new Span<byte>(array1).SequenceEqual(new Span<byte>(array2));
}

void PrintDifferences(byte[] array1, byte[] array2)
{
    if (array1 == null || array2 == null)
    {
        Console.WriteLine("One or both arrays are null.");
        return;
    }

    int length = Math.Min(array1.Length, array2.Length);

    for (int i = 0; i < length; i++)
    {
        if (array1[i] != array2[i])
        {
            Console.WriteLine($"Difference at index {i}: array1 = {array1[i]}, array2 = {array2[i]}");
        }
    }

    if (array1.Length != array2.Length)
    {
        Console.WriteLine("The arrays have different lengths.");
        if (array1.Length > array2.Length)
        {
            for (int i = length; i < array1.Length; i++)
            {
                Console.WriteLine($"Extra byte in array1 at index {i}: {array1[i]}");
            }
        }
        else
        {
            for (int i = length; i < array2.Length; i++)
            {
                Console.WriteLine($"Extra byte in array2 at index {i}: {array2[i]}");
            }
        }
    }
}

Installed Packages ConsoleTables, 2.6.1